# Redes Neuronales
## Preparacion de datos para el modelo

In [1]:
import sys
sys.path.append('../')

import Utilidades as ut
import Modelos as md
import numpy as np
import pandas as pd


df_train = pd.read_pickle("../Archivos/Neuronales_entrenamiento.pkl")
df_test = pd.read_pickle("../Archivos/Neuronales_validacion.pkl")

if ('Opportunity_ID' in df_test):
    df_test = df_test.drop(columns=['Opportunity_ID'])

####MOMENTANEO

#df_train = df_train.drop(columns=['Opportunity_Created_Date'])
#df_test = df_test.drop(columns=['Opportunity_Created_Date'])

##############3
    
x_train, y_train = ut.split_labels(df_train)
x_test, y_test = ut.split_labels(df_test)

#Convertimos las fechas a numeros (cantidad de dias transcurridos) y luego las normalizamos
x_train, x_test = ut.conversion_fechas(x_train, x_test)
x_train, x_test = ut.codificar_categoricas(x_train, y_train, x_test, modo='catboost')
x_train, x_test = ut.normalizacion_numericas(x_train, x_test, modo='normalizacion')
x_train.shape

/home/julian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


(12140, 6)

## Creacion del modelo.

In [2]:
import keras
import tensorflow as tf
from keras import backend
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from sklearn.metrics import accuracy_score, log_loss


x_train_vector = ut.df_a_vector(x_train)
y_train_vector = ut.df_a_vector(y_train)
x_test_vector = ut.df_a_vector(x_test)
y_test_vector = ut.df_a_vector(y_test)

#input_dim = x_train.shape[1]
#
#alfa = 0.01
#
#model = Sequential()
#model.add(Dense(96, input_dim=input_dim, kernel_regularizer=l1(alfa), activation='tanh'))
#model.add(Dropout(0.39))
#model.add(Dense(54, kernel_regularizer=l1(alfa), activation='tanh'))
#model.add(Dropout(0.28))
##model.add(Dense(256, kernel_regularizer=l1(alfa), bias_regularizer=l1(alfa), activation='tanh'))
#model.add(Dropout(0.5))
#model.add(Dense(128, kernel_regularizer=l1(alfa), bias_regularizer=l1(alfa), activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(64, kernel_regularizer=l1(alfa), bias_regularizer=l1(alfa), activation='tanh'))
#model.add(Dropout(0.25))
#model.add(Dense(16, kernel_regularizer=l1(alfa), bias_regularizer=l1(alfa), activation='relu'))
#model.add(Dense(8, kernel_regularizer=l1(alfa), bias_regularizer=l1(alfa), activation='relu'))
#model.add(Dense(1, activation='sigmoid'))


#model.compile(loss='binary_crossentropy', optimizer='rmsprop')

#backend.set_value(model.optimizer.learning_rate, 0.0016)

best_hparams = ut.hyperparams_from_json('../Archivos/Neuronales')
model = md.get_neural_network_model(best_hparams, x_train.shape[1])

epochs = 250
batch_size = 256


my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint("Neuronales_Mejor_Modelo.hdf5", 
                                       monitor='val_loss', 
                                       verbose=0,
                                       save_best_only=True, 
                                       mode='min'),
    
    tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                     min_delta=0.0001,
                                     mode='min',
                                     patience=10),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                      mode='min',
                                      factor=0.5,
                                      min_delta=0.0001,
                                      patience=2,
                                      cooldown=0, 
                                      min_lr=1e-24)
]

#def train(last_best):
#    
#    for a in range(0, 50):
#        model = md.get_neural_network_model(best_hparams, x_train.shape[1])
#        history = model.fit(x_train_vector,
#                            y_train_vector,
#                            validation_data=(x_test_vector, y_test_vector),
#                            **fit_dict)
#        if history.history['val_loss'][-1] < last_best:
#            return
#        
#    
#
#train(0.281)
#Mejor modelo hasta el momento val_loss = 0.311
#Ultimo mejor 0.2809
last_best = 0.200

tot = 25

# for a in range(1, tot+1):
#     print(f"Iteracion numero {a} de {tot}")
#     model = md.get_neural_network_model(best_hparams, x_train.shape[1])
#     #backend.set_value(model.optimizer.learning_rate, 1e-4)
#     history = model.fit(x_train_vector,
#                         y_train_vector,
#                         validation_data=(x_test_vector, y_test_vector),
#                         epochs=epochs,
#                         batch_size=batch_size,
#                         verbose=0,
#                         callbacks=my_callbacks)
#     new_model = keras.models.load_model('Neuronales_Mejor_Modelo.hdf5')
#     y_pred_proba = new_model.predict(x_test_vector).flatten()
#     score = log_loss(y_test_vector.flatten(), y_pred_proba, eps=1e-7)
#     if score < last_best:
#         print(f"Se ha conseguido un score mejor al anterior. log_loss = {score}")
#         score_redondeado = round(score, 4)
#         print("Guardando modelo en: 'Mejores_Modelos/Neuronales_logloss_" + str(score_redondeado) + "_10_features.hdf5")
#         new_model.save('Mejores_Modelos/Neuronales_logloss_' + str(score_redondeado) + '_10_features.hdf5')
#         print(f"Best log_loss = {score}")
#         last_best = score

#backend.set_value(model.optimizer.learning_rate, 1e-4)

history = model.fit(x_train_vector,
                       y_train_vector,
                       validation_data=(x_test_vector, y_test_vector),
                       epochs=epochs,
                       batch_size=batch_size,
                       verbose=1,
                       callbacks=my_callbacks)

Cargando hiperparametros desde el archivo: '../Archivos/Neuronales_best_hyperparam.json'
Epoch 1/250


KeyboardInterrupt: 

## Resultados

In [3]:
#new_model = keras.models.load_model('Neuronales_Mejor_Modelo.hdf5')
new_model = keras.models.load_model('Neuronales_logloss_1937_5_features.hdf5')
#new_model.save('Neuronales_logloss_1937_5_features.hdf5')
#new_model = keras.models.load_model('Neuronales_Mejor_Modelo.hdf5')
#new_predictions = new_model.predict(x_test)

In [4]:
from sklearn.metrics import accuracy_score, log_loss

y_pred_proba = new_model.predict(x_test_vector).flatten()
y_pred = [a > 0.5 for a in y_pred_proba]


#Mejor acc obtenido 0.94839
#Mejor log_loss obtenido 0.18612
#Mejor log_loss obtenido que resulto en un buen score en Kaggle -> 0.19372

acc = accuracy_score(y_pred, y_test_vector)
loss = log_loss(y_test_vector.flatten(), y_pred_proba, eps=1e-7)
print(f"acc: {acc}")
print(f"log_loss: {loss}")

acc: 0.9483942414174973
log_loss: 0.1937216449100916


In [ ]:
#Probamos normalizar al intervalo (0, 1)
vector = y_pred_proba.flatten()

min_ = vector.min()
max_ = vector.max()
div = max_ - min_

for i, a in enumerate(vector):
    vector[i] = (a - min_) / div

    
score = log_loss(y_test_vector.flatten(), vector, eps=1e-7)
score

In [ ]:
new_model.evaluate(x_test_vector, y_test_vector)#, metrics=['accuracy'])

In [ ]:
#Podemos analizar las filas que originaron los errores de prediccion

analysis_df = y_test.copy()
analysis_df['Predicted'] = pd.Series(y_pred).astype(int)
differences = (analysis_df['Predicted'] != analysis_df['Stage'])
df_test[differences]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['val_loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Accuracy vs Epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()